In [2]:
import cv2
import numpy as np
import pandas as pd
import csv
import os

# Fungsi untuk menentukan label berdasarkan folder tempat gambar berada
def get_label_from_parent_folder(folder_name):
    if 'gabah beras merah' in folder_name.lower():
        return 'gabah_beras_merah'
    elif 'gabah bangkok' in folder_name.lower():
        return 'gabah_bangkok'
    elif 'gabah ir64' in folder_name.lower():
        return 'gabah_ir64'
    else:
        return None  # Kembalikan None jika tidak ada label yang sesuai

def get_quality_from_subfolder(subfolder_name):
    if 'utuh' in subfolder_name.lower():
        return 'utuh'
    elif 'patah' in subfolder_name.lower():
        return 'patah'
    elif 'rusak' in subfolder_name.lower():
        return 'rusak'
    else:
        return None  # Kembalikan None jika tidak ada label yang sesuai

def extract_features(img_path, parent_label, kualitas_gabah):
    try:
        # Membaca gambar
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f"Gambar {img_path} tidak dapat dibaca.")
    except Exception as e:
        print(f"Error pada {image_path}: {e}")
        return None  # Mengembalikan None jika ada error

    # Konversi ke grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Deteksi tepi Canny
    edges = cv2.Canny(gray, 100, 200)

    # Temukan kontur objek
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Ekstraksi fitur bentuk dan ukuran
    area = 0
    for cnt in contours:
        # Hitung luas area
        area += cv2.contourArea(cnt)
            
        # Hitung perimeter
        perimeter = cv2.arcLength(cnt, True)
            
        # Hitung circularity
        if perimeter == 0:
            circularity = 0  
        else:
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            
        # Hitung aspect ratio
        x, y, w, h = cv2.boundingRect(cnt)
        if h == 0:
            aspect_ratio = 0  
        else:
            aspect_ratio = float(w) / h
                
    features = {
        'File': os.path.basename(img_path),
        'Area': area,
        'Perimeter': perimeter,
        'Circularity': circularity,
        'Aspect Ratio': aspect_ratio,
        'Jenis': parent_label,
        'Kualitas': kualitas_gabah
    }
    return features

# Menyimpan fitur ke dalam DataFrame dan CSV
def save_features_to_csv(input_folder, output_csv):
    data = []
    
    # Iterasi melalui subfolder (gabah beras merah, gabah bangkok, ir64) di dalam folder input
    for parent_folder_name in os.listdir(input_folder):
        parent_folder_path = os.path.join(input_folder, parent_folder_name)
        
        # Pastikan itu adalah folder dan bukan file
        if os.path.isdir(parent_folder_path):
            # Tentukan label parent gabah berdasarkan nama folder
            parent_label = get_label_from_parent_folder(parent_folder_name)
            if parent_label is None:
                print(f"Folder {parent_folder_name} tidak memiliki label yang sesuai, lewati.")
                continue
            
            # Iterasi melalui subfolder (utuh, patah, rusak) di dalam folder parent
            for subfolder_name in os.listdir(parent_folder_path):
                subfolder_path = os.path.join(parent_folder_path, subfolder_name)
                
                # Pastikan itu adalah folder dan bukan file
                if os.path.isdir(subfolder_path):
                    # Tentukan label kualitas gabah berdasarkan nama subfolder
                    quality_label = get_quality_from_subfolder(subfolder_name)
                    if quality_label is None:
                        print(f"Subfolder {subfolder_name} tidak memiliki label yang sesuai, lewati.")
                        continue
                    
                    # Input manual untuk 'jenis' gabah
                    kualitas_gabah = input(f"Masukkan kualitas gabah untuk folder '{subfolder_name}' '{parent_folder_name}': ")

                    # Iterasi melalui file gambar dalam subfolder
                    for filename in os.listdir(subfolder_path):
                        file_path = os.path.join(subfolder_path, filename)
                        
                        # Memeriksa ekstensi file untuk memastikan itu adalah gambar
                        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                            print(f"File {filename} bukan gambar, lewati.")
                            continue
                        
                        # Ekstrak fitur dari gambar
                        features = extract_features(file_path, parent_label, kualitas_gabah)
                        if features:  # Hanya menambahkan fitur yang valid
                            data.append(features)
                        else:
                            print(f"Gambar {filename} gagal diproses.")

    # Membuat DataFrame dari data
    df = pd.DataFrame(data)
    
    # Menyimpan DataFrame ke CSV
    df.to_csv(output_csv, index=False)
    print(f"Data berhasil disimpan dalam {output_csv}")

# Direktori gambar (folder utama yang berisi subfolder gabah beras merah, gabah bangkok, ir64)
input_folder = r'C:\Users\Naufal\OneDrive\MyDocuments\KULIAH\Tugas Kuliah\TUGAS AKHIR'  # Ganti dengan direktori gambar Anda
output_csv = 'gabah_shape_features.csv'

save_features_to_csv(input_folder, output_csv)
print("Hasil ekstraksi fitur telah disimpan ke file gabah_shape_features.csv")

Folder berkas daftar sidang tidak memiliki label yang sesuai, lewati.
Folder dataset tidak memiliki label yang sesuai, lewati.


Masukkan kualitas gabah untuk folder 'patah' 'gabah bangkok':  patah
Masukkan kualitas gabah untuk folder 'rusak' 'gabah bangkok':  rusak
Masukkan kualitas gabah untuk folder 'utuh' 'gabah bangkok':  utuh
Masukkan kualitas gabah untuk folder 'patah' 'gabah beras merah':  patah
Masukkan kualitas gabah untuk folder 'rusak' 'gabah beras merah':  rusak
Masukkan kualitas gabah untuk folder 'utuh' 'gabah beras merah':  utuh
Masukkan kualitas gabah untuk folder 'patah' 'gabah ir64':  patah
Masukkan kualitas gabah untuk folder 'rusak' 'gabah ir64':  rusk
Masukkan kualitas gabah untuk folder 'utuh' 'gabah ir64':  utuh


Folder referensi tidak memiliki label yang sesuai, lewati.
Data berhasil disimpan dalam gabah_shape_features.csv
Hasil ekstraksi fitur telah disimpan ke file gabah_shape_features.csv
